In [7]:
import pandas as pd
import ast
from funciones import extract
import pyarrow
import json
import gzip
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
%load_ext memory_profiler
import pyarrow.parquet as pq


The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [8]:
df = pd.read_csv(r'data_set_limpio/games_preparado.csv.gz')

In [10]:
opinion = pd.read_csv(r'data_set_limpio/reviews_preparado.csv.gz')

In [11]:
item_user = pd.read_csv(r'data_set_limpio\item_cantidad_usuarios.csv.gz')

In [4]:

def developer(developer: str):
    if developer not in list(df['developer']):
        

    fechas = df['release_date'].unique()

    anio = {}
    free = {}

    for x in fechas:
        filter_condition = (df['release_date'] == x) & (df['developer'] == developer)
        developer_releases = df[filter_condition]

        if len(developer_releases) != 0:
            anio[x] = len(developer_releases)
            free[x] = len(developer_releases[developer_releases['price'] == 0.0])

    for x, y in free.items():
        free[x] = f"{round((y / anio[x]) * 100, 2)}%"

    # Convertir a tipos de datos nativos de Python
    anio = {str(k): v for k, v in anio.items()}
    free = {str(k): v for k, v in free.items()}

    resultado = {'Cantidad de Items': anio, 'Contenido Free': free}

    # Serializar a JSON usando json.dumps
    return resultado


In [5]:
developer('Valve')

{'Cantidad de Items': {'2017': 2,
  '1998': 1,
  '2006': 2,
  '2003': 1,
  '2007': 3,
  '2000': 2,
  '2001': 1,
  '2004': 5,
  '2008': 1,
  '2009': 1,
  '1999': 1,
  '2010': 2,
  '2011': 1,
  '2012': 1},
 'Contenido Free': {'2017': '0.0%',
  '1998': '0.0%',
  '2006': '0.0%',
  '2003': '0.0%',
  '2007': '33.33%',
  '2000': '0.0%',
  '2001': '0.0%',
  '2004': '0.0%',
  '2008': '0.0%',
  '2009': '0.0%',
  '1999': '0.0%',
  '2010': '50.0%',
  '2011': '0.0%',
  '2012': '0.0%'}}

In [9]:
def userdata(user: str):
    try:
        usuario = 0
        chunks = pd.read_csv(r'data_set_limpio\item_desplegado.csv.gz', chunksize=200000)
        for chunk in chunks:
            if chunk['user'].isin([user]).any():
                usuario = chunk[chunk['user'] == user]
                break

        resultado = {
            'Usuario' : user,
            'Dinero gastado' : str(usuario['price'].sum())+'$',
            'cantidad de items' : str(item_user.loc[item_user['user_id'] == user]['items_count'].values[0]),
            'Porcentaje de recomendaciones' :str(round((len(opinion.loc[opinion['user'] == user])  / item_user.loc[item_user['user_id'] == user]['items_count'].values[0])*100,2))+'%'}
        
        return resultado
    except:
        return 'El usuario no se encuentra en la base de datos.'

In [10]:
userdata('76561197970982479')

{'Usuario': '76561197970982479',
 'Dinero gastado': '2315 USD',
 'cantidad de items': '277',
 'Porcentaje de recomendaciones': '1.08%'}

In [11]:
def best_developer_year(year : int):
    
        anio = pd.read_csv(r'data_set_limpio//Max_developer_year.csv')

            
        anio = anio.sort_values('Anio', ascending=False)
        
        seleccion = anio.loc[anio['Anio'] == year].copy()
        
        seleccion.fillna('Sin informacion',inplace=True)
        
        if not seleccion['Anio'].empty:
            respuesta = {'Anio': str(seleccion['Anio'].values[0]),
                        'Top 1': str(seleccion['Top 1'].values[0]),
                        'Top 2': str(seleccion['Top 2'].values[0]),
                        'Top 3': str(seleccion['Top 3'].values[0])
                        }
        else:
            respuesta = {f'No ingreso un valor relevante, este es el rango disponible ({str(anio["Anio"].min())} - {str(anio["Anio"].max())})'}

        return respuesta

In [12]:
best_developer_year(2017)

{'Anio': '2017',
 'Top 1': "('Smartly Dressed Games', 594)",
 'Top 2': "('Freejam', 178)",
 'Top 3': "('Studio Wildcard,Instinct Games,Efecto Studios,Virtual Basement LLC', 160)"}

In [12]:
def resenias_developer( des : str ):
    
    try:

        table = pq.read_table(r'data_set_limpio/recomends_dev.parquet')
        
        dev = table.to_pandas()
        
        developer = dev.loc[dev['Developers'] == des]
        
        respuesta = {'Deloper' : str(developer['Developers'].values[0]), 'Reviews positivos' : str(developer['Positivo'].values[0]),'Reviews negativos' : str(developer['Negativo'].values[0])}
        dev = 0
        return respuesta
    
    except:
        
        return 'No ingreso un valor relevante, o el desarrollador no se encuentra en la base de datos'   

In [14]:
resenias_developer('Freejam')

{'Deloper': 'Freejam', 'Reviews positivos': '211', 'Reviews negativos': '71'}